<a href="https://colab.research.google.com/github/Siliconvalley4uYouthProjects/SpringBoard-Swatcloud/blob/main/Recommendation_system_content_based_with_wanted_unwanted_keywords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Importing libraries

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# Reading the file with the keyword field as string

df = pd.read_csv('jobs_wanted_unwanted_keywords_string.csv')

In [4]:
# Checking for null values
df.isna().sum()

Company                  0
Title                    0
Technical Keywords       0
Nontechnical Keywords    0
dtype: int64

In [5]:
# get the index of id
def index_from_id(df,id):

 return df[df.index==id].index.values[0]

# id: the one you want to match, it can be student id or company id, if it is student, then retrieve all companys skillset, and create a dataframe,(match company for a given student)
# and if it is company id, then retrieve all students skillset, and create a dataframe(match students for a given company)
# df: dataframe needed to be make
#cosine_similarity_matrix: converted from dataframe[keyword] column
#number_of_recommendations: the number you want to match
def recommendations( id, df,cosine_similarity_matrix,number_of_recommendations):

  index = index_from_id(df,id)

  similarity_scores = list(enumerate(cosine_similarity_matrix[index]))

  similarity_scores_sorted = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

  recommendations_indices = [t[0] for t in similarity_scores_sorted[1:(number_of_recommendations+1)]]
  # this could be df['ID'] 
  return df['Title'].iloc[recommendations_indices]

In [6]:
vect = CountVectorizer(stop_words='english')
# transform df['Technical Keywords'] into matrix
vect_matrix = vect.fit_transform(df['Technical Keywords'])
vect_matrix
# get the cosine_similarity_matrix
cosine_similarity_matrix_count_based = cosine_similarity(vect_matrix, vect_matrix)

In [21]:
#Testing the function
recommendations(10,df,cosine_similarity_matrix_count_based,5)

42           Sr Software Dev Engineer, Business Payments
13                         Software Development Engineer
31                  Software Development Engineer , CTPS
249    Sr. Software Development Engineer - AWS DNS, A...
2                          Software Development Engineer
Name: Title, dtype: object

In [8]:
# Manually checking if the function works as expected
# Taking the keyword string in the 10th index value as the qualifications of an applicant.

similarity_scores = list(enumerate(cosine_similarity_matrix_count_based[10]))

# Sort similarity scores.
# Using index [1:6] for top 5 similar because the one in the 0 index value 
# is the key word string itself we are trying to match.

similarity_scores_sorted = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
similarity_scores_sorted[1:6]

[(42, 0.9649012813540149),
 (13, 0.947514248563855),
 (31, 0.9468641529479991),
 (249, 0.9291641968594219),
 (2, 0.8934271123648286)]

In [9]:
# The recommended jobs

recommendations_indices = [t[0] for t in similarity_scores_sorted[1:6]]
df[['Title', 'Technical Keywords']].iloc[recommendations_indices]

,Title,Technical Keywords
42,"Sr Software Dev Engineer, Business Payments","['', 'language', 'development', 'ui', 'design'..."
13,Software Development Engineer,"['', 'language', 'development', 'ui', 'design'..."
31,"Software Development Engineer , CTPS","['', 'language', 'development', 'ui', 'managem..."
249,"Sr. Software Development Engineer - AWS DNS, A...","['', 'language', 'development', 'ui', 'design'..."
2,Software Development Engineer,"['web', '', 'language', 'development', 'ui', '..."


These recommendations match with the output that the recommendations function had above.


In [10]:
# the job that was taken as an applicant's qualification

print(df['Title'].iloc[10])
print(df['Technical Keywords'].iloc[10])

Software Engineer
['', 'language', 'development', 'ui', 'design', 'management', 'r', 'software', 'engineering', 'systems', 'computer', 'code', 'application', 'engineer', 'programming', 'architecture', 'communication', 'testing', 'agile', 'coding', 'scaling', 'software development', 'computer science']


In [11]:
# The keywords that were listed in the qualification requirements of the recommended jobs

for i in recommendations_indices:
    print(df['Technical Keywords'].iloc[i])

['', 'language', 'development', 'ui', 'design', 'management', 'r', 'software', 'engineering', 'systems', 'computer', 'code', 'engineer', 'programming', 'architecture', 'communication', 'testing', 'coding', 'scaling', 'software development', 'computer science']
['', 'language', 'development', 'ui', 'design', 'management', 'r', 'software', 'engineering', 'systems', 'computer', 'code', 'product', 'engineer', 'programming', 'architecture', 'communication', 'testing', 'coding', 'scaling', 'software development', 'computer science']
['', 'language', 'development', 'ui', 'management', 'r', 'software', 'engineering', 'systems', 'computer', 'code', 'application', 'engineer', 'programming', 'communication', 'testing', 'agile', 'coding', 'software development', 'computer science']
['', 'language', 'development', 'ui', 'design', 'management', 'r', 'software', 'engineering', 'systems', 'computer', 'code', 'engineer', 'programming', 'bachelor', 'architecture', 'testing', 'coding', 'scaling', 'softwa

### The below key words were in the text we are trying to match as well as the 5 recommended jobs. This model seem to pick relevant job descriptions.

    ui design
    management
    software engineering
    product engineer
    architecture
    security
    r

In [15]:
#Testing the algorithm without any indeed jobs
df2 = pd.read_csv('jobs_wanted_unwanted_keywords_without_indeed_string.csv')
vect2 = CountVectorizer(stop_words='english')
# transform df['Technical Keywords'] into matrix
vect_matrix2 = vect2.fit_transform(df2['Technical Keywords'])
# get the cosine_similarity_matrix
cosine_similarity_matrix_count_based2 = cosine_similarity(vect_matrix2, vect_matrix2)

In [20]:
recommendations(10,df,cosine_similarity_matrix_count_based2,5)

42           Sr Software Dev Engineer, Business Payments
13                         Software Development Engineer
31                  Software Development Engineer , CTPS
249    Sr. Software Development Engineer - AWS DNS, A...
2                          Software Development Engineer
Name: Title, dtype: object

The same 5 jobs are recommended with or without the inclusion of jobs listed on indeed. This isn't surprising since all of the recommended jobs in our example were other Amazon jobs anyways.